In [2]:
################# 실제 작동되는 코드 ####################

import paramiko
import os

ip = "172.16.16.222"
user = "root"
pw = "asd123!@"

SERVER_NFS_DIR = ""
CLIENT_NFS_DIR = ""
CLIENT_NFS_NI = "172.16.0.0" ## NFS를 허용해줄 CLIENT의 NFS IP입니다 
SERVER_NFS_IP = "172.16.16.100"

def cc(cmd):
    cli = paramiko.SSHClient()
    cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    cli.connect(ip, username=user, password=pw, port=22)
    stdin, stdout, stderr = cli.exec_command(cmd)
    out = stdout.read().decode()
    err = stderr.read().decode()
    cli.close()
    return out + err
##################################
#          SERVER_NFS            #
##################################

def server_nfs_install():
    cc("apt update; apt install nfs-kernel-server -y")

def server_nfs_mkdir():
    global SERVER_NFS_DIR
    SERVER_NFS_DIR = input("서버가 NFS로 공유해줄 디렉토리를 입력해주세요 ex. /data/nfs_share ")
    cmd =  f"""
mkdir -p {SERVER_NFS_DIR}
chown nobody:nogroup {SERVER_NFS_DIR}
chmod 777 {SERVER_NFS_DIR}
    """
    cc(cmd)

def server_nfs_set_exports_config():
    CLIENT_NFS_NI = input("NFS 공유를 허용할 클라이언트의 NI를 입력해주세요 (예: 172.16.0.0/16): ")
    print(f"클라이언트 {CLIENT_NFS_NI} NI 대역에게 읽기/쓰기 권한을 부여합니다.")
    cc(f"echo '{SERVER_NFS_DIR} {CLIENT_NFS_NI}(rw,sync,no_subtree_check)' >> /etc/exports")
#    cc(f"echo '{SERVER_NFS_DIR} {CLIENT_IP}(rw,sync,no_subtree_check)' >> /etc/exports")

    cc("exportfs -ra ; systemctl restart nfs-kernel-server")
    cc("systemctl stop ufw")

def server_nfs():
    server_nfs_install()
    server_nfs_mkdir()
    server_nfs_set_exports_config()
##################################
#          CLIENT_NFS            #
##################################

def client_nfs_install():
    cc("apt update ; apt install nfs-common -y")

def client_nfs_mkdir():
    global CLIENT_NFS_DIR
    CLIENT_NFS_DIR = input("클라가 NFS로 공유받을 디렉토리를 입력해주세요 ex. /mnt/nfs_share ")
    cmd =  f"""
mkdir -p {CLIENT_NFS_DIR}
    """
    cc(cmd)
def client_nfs_mount_temporarily():
    SERVER_IP = input("NFS 서버 IP를 입력하세요 (ex. 172.16.16.100): ")
    CLIENT_NFS_DIR = input("클라가 NFS로 공유받을 디렉토리를 입력해주세요 ex. /mnt/nfs_share ")
    SERVER_NFS_DIR = input("서버 공유 디렉토리 경로를 입력하세요 (ex. /data/nfs_share): ")
    
    cmd = f"mount {SERVER_IP}:{SERVER_NFS_DIR} {CLIENT_NFS_DIR}"
    cc(cmd)  # cc는 SSH 또는 로컬 명령 실행 함수
    print(cc("df -h | grep nfs"))

def client_nfs():
    client_nfs_install()
    client_nfs_mkdir()
    client_nfs_mount_temporarily()

## 실행부
#server_nfs()
client_nfs()

클라가 NFS로 공유받을 디렉토리를 입력해주세요 ex. /mnt/nfs_share  /mnt/nfs_share
NFS 서버 IP를 입력하세요 (ex. 172.16.16.100):  172.16.16.24
클라가 NFS로 공유받을 디렉토리를 입력해주세요 ex. /mnt/nfs_share  /mnt/nfs_share
서버 공유 디렉토리 경로를 입력하세요 (ex. /data/nfs_share):  /data/nfs_share


172.16.16.24:/data/nfs_share       9.8G  5.1G  4.2G  55% /mnt/nfs_share

